In [140]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [141]:
from sklearn.preprocessing import MinMaxScaler


In [142]:
data = pd.read_csv('CNA.csv')


In [143]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1973-02-21,0.000000,5.666667,5.500000,5.500000,3.902713,45900
1,1973-02-22,0.000000,5.583333,5.500000,5.541667,3.932280,48900
2,1973-02-23,0.000000,5.625000,5.375000,5.375000,3.814015,957900
3,1973-02-26,0.000000,5.458333,5.333333,5.458333,3.873148,54900
4,1973-02-27,0.000000,5.500000,5.375000,5.416667,3.843580,44700
...,...,...,...,...,...,...,...
11879,2020-03-26,31.120001,32.770000,30.620001,32.619999,32.619999,260000
11880,2020-03-27,31.600000,32.830002,29.840000,31.990000,31.990000,313400
11881,2020-03-30,32.029999,33.240002,31.150000,32.980000,32.980000,248500
11882,2020-03-31,32.720001,32.720001,30.879999,31.040001,31.040001,267700


In [144]:
data = data.filter(['Close'])


In [145]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,Close
10884,31.990000
10885,31.790001
10886,32.099998
10887,32.099998
10888,32.299999
...,...
11879,32.619999
11880,31.990000
11881,32.980000
11882,31.040001


In [146]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [147]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [148]:

data = train_data

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]


In [149]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 14
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [150]:
start_time = time.time()

# Tạo một list chứa các số Lucas để sử dụng làm số lượng hidden layer
lucas_list = [3, 4, 7, 11, 18, 29, 47, 76]

# Chọn số lượng hidden layer sử dụng theo số Lucas
num_hidden_layer = lucas_list[7] # ví dụ chọn số 11

In [151]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=4, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(num_hidden_layer): #Thêm num_hidden_layer lớp ẩn với 64 đơn vị mỗi lớp
    model.add(Dense(units=100, activation='relu')) #Tham số chỉ định số lượng tế bào thần kinh trong lớp
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [152]:
history = model.fit(X_train, y_train, epochs=8000, batch_size=8)

Epoch 1/100
40/40 [==============================] - 17s 10ms/step - loss: 0.4225
Epoch 2/100
40/40 [==============================] - 0s 10ms/step - loss: 0.3772
Epoch 3/100
40/40 [==============================] - 0s 11ms/step - loss: 0.3370
Epoch 4/100
40/40 [==============================] - 0s 11ms/step - loss: 0.3000
Epoch 5/100
40/40 [==============================] - 0s 11ms/step - loss: 0.2671
Epoch 6/100
40/40 [==============================] - 0s 11ms/step - loss: 0.2379
Epoch 7/100
40/40 [==============================] - 0s 11ms/step - loss: 0.2124
Epoch 8/100
40/40 [==============================] - 0s 11ms/step - loss: 0.1898
Epoch 9/100
40/40 [==============================] - 0s 11ms/step - loss: 0.1697
Epoch 10/100
40/40 [==============================] - 0s 11ms/step - loss: 0.1527
Epoch 11/100
40/40 [==============================] - 0s 11ms/step - loss: 0.1377
Epoch 12/100
40/40 [==============================] - 0s 11ms/step - loss: 0.1246
Epoch 13/100
40/40 [====

In [153]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

5/5 [==============================] - 1s 4ms/step
Training time: 64.2109169960022 seconds


In [154]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAPE: ', mape)

5/5 [==============================] - 0s 3ms/step
MSE:  0.002484723738781816
RMSE:  0.04984700330794035
MAPE:  0.0646953394197951


In [155]:
len(data)

800